# Supervisor (Polyglot)

In [ ]:
#r @"../../../../../../../.nuget/packages/fsharp.control.asyncseq/3.2.1/lib/netstandard2.1/FSharp.Control.AsyncSeq.dll"
#r @"../../../../../../../.nuget/packages/system.reactive/6.0.1-preview.1/lib/net6.0/System.Reactive.dll"
#r @"../../../../../../../.nuget/packages/system.reactive.linq/6.0.1-preview.1/lib/netstandard2.0/System.Reactive.Linq.dll"
#r @"../../../../../../../.nuget/packages/argu/6.2.4/lib/netstandard2.0/Argu.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.http.connections.common/7.0.0/lib/net7.0/Microsoft.AspNetCore.Http.Connections.Common.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.http.connections.client/7.0.0/lib/net7.0/Microsoft.AspNetCore.Http.Connections.Client.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.signalr.common/7.0.0/lib/net7.0/Microsoft.AspNetCore.SignalR.Common.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.signalr.client/7.0.0/lib/net7.0/Microsoft.AspNetCore.SignalR.Client.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.signalr.client.core/7.0.0/lib/net7.0/Microsoft.AspNetCore.SignalR.Client.Core.dll"
#r @"../../../../../../../.nuget/packages/fsharp.json/0.4.1/lib/netstandard2.0/FSharp.Json.dll"

In [ ]:
#!import ../../lib/fsharp/Notebooks.dib
#!import ../../lib/fsharp/Testing.dib

In [ ]:
#!import ../../lib/fsharp/Common.fs
#!import ../../lib/fsharp/CommonFSharp.fs
#!import ../../lib/fsharp/Async.fs
#!import ../../lib/fsharp/AsyncSeq.fs
#!import ../../lib/fsharp/Runtime.fs
#!import ../../lib/fsharp/FileSystem.fs

In [ ]:
#if !INTERACTIVE
open Lib
#endif

In [ ]:
open Common
open SpiralFileSystem.Operators
open Microsoft.AspNetCore.SignalR.Client

### sendJson

In [ ]:
let sendJson (port : int) (json : string) = async {
    let host = "127.0.0.1"
    let! portOpen = SpiralNetworking.test_port_open host port
    if portOpen then
        try
            let connection = HubConnectionBuilder().WithUrl($"http://{host}:{port}").Build()
            do! connection.StartAsync () |> Async.AwaitTask
            let! result = connection.InvokeAsync<string> ("ClientToServerMsg", json) |> Async.AwaitTask
            do! connection.StopAsync () |> Async.AwaitTask
            trace Verbose (fun () -> $"Supervisor.sendJson / port: {port} / json: {json |> SpiralSm.ellipsis_end 200} / result: {result |> Option.ofObj |> Option.map (SpiralSm.ellipsis_end 200)}") _locals
            return Some result
        with ex ->
            trace Critical (fun () -> $"Supervisor.sendJson / port: {port} / json: {json |> SpiralSm.ellipsis_end 200} / ex: {ex |> SpiralSm.format_exception}") _locals
            return None
    else
        trace Debug (fun () -> $"Supervisor.sendJson / port: {port} / error: port not open") _locals
        return None
}

### sendObj

In [ ]:
let sendObj port obj =
    obj
    |> System.Text.Json.JsonSerializer.Serialize
    |> sendJson port

### VSCPos

In [ ]:
type VSCPos = {| line : int; character : int |}

### VSCRange

In [ ]:
type VSCRange = VSCPos * VSCPos

### RString

In [ ]:
type RString = VSCRange * string

### TracedError

In [ ]:
type TracedError = {| trace : string list; message : string |}

### ClientErrorsRes

In [ ]:
type ClientErrorsRes =
    | FatalError of string
    | TracedError of TracedError
    | PackageErrors of {| uri : string; errors : RString list |}
    | TokenizerErrors of {| uri : string; errors : RString list |}
    | ParserErrors of {| uri : string; errors : RString list |}
    | TypeErrors of {| uri : string; errors : RString list |}

### workspaceRoot

In [ ]:
let workspaceRoot = SpiralFileSystem.get_workspace_root ()

### awaitCompiler

In [ ]:
let awaitCompiler port cancellationToken = async {
    let host = "127.0.0.1"
    let struct (ct, disposable) = cancellationToken |> SpiralThreading.new_disposable_token
    let! ct = ct |> SpiralAsync.merge_cancellation_token_with_default_async

    let compiler = MailboxProcessor.Start (fun inbox -> async {
        let! availablePort = SpiralNetworking.get_available_port (Some 180) host port
        if availablePort <> port then
            inbox.Post (port, false)
        else
            let compilerPath =
                workspaceRoot </> "deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release"
                |> System.IO.Path.GetFullPath

            let dllPath = compilerPath </> "Spiral.dll"

            let! exitCode, result =
                SpiralRuntime.execution_options (fun x ->
                    { x with
                        l0 = $@"dotnet ""{dllPath}"" --port {availablePort} --default-int i32 --default-float f64"
                        l1 = Some ct
                        l3 = Some (fun struct (_, line, _) -> async {
                            if line |> SpiralSm.contains $"System.IO.IOException: Failed to bind to address http://{host}:{port}: address already in use." then
                                inbox.Post (port, false)

                            if line |> SpiralSm.contains $"Server bound to: http://localhost:{availablePort}" then
                                let rec loop retry = async {
                                    do!
                                        SpiralNetworking.wait_for_port_access (Some 100) true host availablePort
                                        |> Async.runWithTimeoutAsync 500
                                        |> Async.Ignore

                                    let _locals () = $"port: {availablePort} / retry: {retry} / {_locals ()}"
                                    try
                                        let pingObj = {| Ping = true |}
                                        let! pingResult = pingObj |> sendObj availablePort
                                        trace Verbose (fun () -> $"Supervisor.awaitCompiler / Ping / result: '%A{pingResult}'") _locals

                                        match pingResult with
                                        | Some _ -> inbox.Post (availablePort, true)
                                        | None -> do! loop (retry + 1)
                                    with ex ->
                                        trace Verbose (fun () -> $"Supervisor.awaitCompiler / Ping / ex: {ex |> SpiralSm.format_exception}") _locals
                                        do! loop (retry + 1)
                                }
                                do! loop 1
                        })
                        l6 = Some workspaceRoot
                    }
                )
                |> SpiralRuntime.execute_with_options_async

            trace Debug (fun () -> $"Supervisor.awaitCompiler / exitCode: {exitCode} / result: {result}") _locals
            disposable.Dispose ()
    }, ct)

    let! serverPort, managed = compiler.Receive ()

    let connection = HubConnectionBuilder().WithUrl($"http://{host}:{serverPort}").Build ()
    do! connection.StartAsync () |> Async.AwaitTask

    let event = Event<_> ()
    let disposable' = connection.On<string> ("ServerToClientMsg", event.Trigger)
    let stream =
        FSharp.Control.AsyncSeq.unfoldAsync
            (fun () -> async {
                let! msg = event.Publish |> Async.AwaitEvent
                return Some (msg |> FSharp.Json.Json.deserialize<ClientErrorsRes>, ())
            })
            ()

    let disposable' =
        new_disposable (fun () ->
            async {
                disposable'.Dispose ()
                do! connection.StopAsync () |> Async.AwaitTask
                disposable.Dispose ()
                if managed
                then do!
                    SpiralNetworking.wait_for_port_access (Some 100) false host serverPort
                    |> Async.runWithTimeoutAsync 1500
                    |> Async.Ignore
            }
            |> Async.RunSynchronously
        )

    return
        serverPort,
        stream,
        ct,
        disposable'
}

### getFilePathFromUri

In [ ]:
let getFilePathFromUri uri =
    match System.Uri.TryCreate (uri, System.UriKind.Absolute) with
    | true, uri -> uri.AbsolutePath |> System.IO.Path.GetFullPath
    | _ -> failwith "invalid uri"

### getCompilerPort

In [ ]:
let getCompilerPort () =
    13805

### serialize_obj

In [ ]:
    let serializeObj obj =
        try
            obj
            |> FSharp.Json.Json.serialize
            |> SpiralSm.replace "\\\\" "\\"
            |> SpiralSm.replace "\\r\\n" "\n"
            |> SpiralSm.replace "\\n" "\n"
        with ex ->
            trace Critical (fun () -> "Supervisor.serialize_obj / obj: %A{obj}") _locals
            ""

### Backend

In [ ]:
type Backend =
    | Fsharp
    | Cuda

### buildFile

In [ ]:
let buildFile backend timeout port cancellationToken path =
    let rec loop retry = async {
        let fullPath = path |> System.IO.Path.GetFullPath |> SpiralFileSystem.normalize_path
        let fileDir = fullPath |> System.IO.Path.GetDirectoryName
        let fileName = fullPath |> System.IO.Path.GetFileNameWithoutExtension
        let! code = fullPath |> SpiralFileSystem.read_all_text_async

        let stream, disposable = fileDir |> FileSystem.watchDirectory (fun _ -> true)
        use _ = disposable

        let struct (token, disposable) = SpiralThreading.new_disposable_token cancellationToken
        use _ = disposable

        let port = port |> Option.defaultWith getCompilerPort
        let! serverPort, errors, ct, disposable = awaitCompiler port (Some token)
        use _ = disposable

        let outputFileName =
            match backend with
            | Fsharp -> $"{fileName}.fsx"
            | Cuda -> $"{fileName}.py"

        let outputContentSeq =
            stream
            |> FSharp.Control.AsyncSeq.chooseAsync (function
                | _, (FileSystem.FileSystemChange.Changed (path, Some text))
                    when (path |> System.IO.Path.GetFileName) = outputFileName
                    ->
                        // fileDir </> path |> SpiralFileSystem.read_all_text_retry_async
                        text |> Some |> Async.init
                | _ -> None |> Async.init
            )
            |> FSharp.Control.AsyncSeq.map (fun content ->
                Some (content |> SpiralSm.replace "\r\n" "\n"), None
            )

        let printErrorData (data : {| uri : string; errors : RString list |}) =
            let fileName = data.uri |> System.IO.Path.GetFileName
            let errors =
                data.errors
                |> List.map snd
                |> SpiralSm.concat "\n"
            $"{fileName}:\n{errors}"

        let errorsSeq =
            errors
            |> FSharp.Control.AsyncSeq.choose (fun error ->
                match error with
                | FatalError message ->
                    Some (message, error)
                | TracedError data ->
                    Some (data.message, error)
                | PackageErrors data when data.errors |> List.isEmpty |> not ->
                    Some (data |> printErrorData, error)
                | TokenizerErrors data when data.errors |> List.isEmpty |> not ->
                    Some (data |> printErrorData, error)
                | ParserErrors data when data.errors |> List.isEmpty |> not ->
                    Some (data |> printErrorData, error)
                | TypeErrors data when data.errors |> List.isEmpty |> not ->
                    Some (data |> printErrorData, error)
                | _ -> None
            )
            |> FSharp.Control.AsyncSeq.map (fun (message, error) ->
                None, Some (message, error)
            )

        let timerSeq =
            500
            |> FSharp.Control.AsyncSeq.intervalMs
            |> FSharp.Control.AsyncSeq.map (fun _ -> None, None)

        let outputSeq =
            [ outputContentSeq; errorsSeq; timerSeq ]
            |> FSharp.Control.AsyncSeq.mergeAll

        let! outputChild =
            ((None, [], 0), outputSeq)
            ||> FSharp.Control.AsyncSeq.scan (
                fun (outputContentResult, errors, typeErrorCount) (outputContent, error) ->
                    trace Debug (fun () -> $"Supervisor.buildFile / AsyncSeq.scan / path: {path} / errors: {errors |> serializeObj} / outputContentResult: {outputContentResult} / typeErrorCount: {typeErrorCount} / retry: {retry} / error: {error} / outputContent:\n{outputContent |> Option.defaultValue System.String.Empty |> SpiralSm.ellipsis_end 1500}") _locals
                    match outputContent, error with
                    | Some outputContent, None -> Some outputContent, errors, typeErrorCount
                    | None, Some (_, FatalError "File main has a type error somewhere in its path.") ->
                        outputContentResult, errors, typeErrorCount + 1
                    | None, Some error -> outputContentResult, error :: errors, typeErrorCount
                    | None, None when typeErrorCount >= 1 ->
                        outputContentResult, errors, typeErrorCount + 1
                    | _ -> outputContentResult, errors, typeErrorCount
            )
            |> FSharp.Control.AsyncSeq.takeWhileInclusive (fun (outputContent, errors, typeErrorCount) ->
                trace Debug (fun () -> $"Supervisor.buildFile / takeWhileInclusive / path: {path} / errors: {errors |> serializeObj} / typeErrorCount: {typeErrorCount} / retry: {retry} / outputContent:\n{outputContent |> Option.defaultValue System.String.Empty |> SpiralSm.ellipsis_end 1500}") _locals
    #if INTERACTIVE
                let errorWait = 2
    #else
                let errorWait = 2
    #endif
                match outputContent, errors with
                | None, [] when typeErrorCount > errorWait -> false
                | None, [] -> true
                | _ -> false
            )
            |> FSharp.Control.AsyncSeq.tryLast
            |> Async.withCancellationToken ct
            |> Async.catch
            |> Async.runWithTimeoutAsync timeout
            |> Async.StartChild

        // do! Async.Sleep 60

        let fullPathUri = fullPath |> SpiralFileSystem.normalize_path |> SpiralFileSystem.new_file_uri

        let fileOpenObj = {| FileOpen = {| uri = fullPathUri; spiText = code |} |}
        let! _fileOpenResult = fileOpenObj |> sendObj serverPort

        // do! Async.Sleep 60

        let backendId =
            match backend with
            | Fsharp -> "Fsharp"
            | Cuda -> "Python + Cuda"
        let buildFileObj = {| BuildFile = {| uri = fullPathUri; backend = backendId |} |}
        let! _buildFileResult = buildFileObj |> sendObj serverPort

        let! result, typeErrorCount =
            outputChild
            |> Async.map (function
                | Some (Ok (Some (outputCode, errors, typeErrorCount))) ->
                    (outputCode, errors |> List.distinct |> List.rev), typeErrorCount
                | Some (Error ex) ->
                    trace Critical (fun () -> $"Supervisor.buildFile / error: {ex |> SpiralSm.format_exception} / retry: {retry}") _locals
                    (None, []), 0
                | _ -> (None, []), 0
            )

        match result with
        | None, _ when typeErrorCount > 0 && retry = 0 ->
            return! loop (retry + 1)
        | _ ->
            if fileDir |> SpiralSm.starts_with (workspaceRoot </> "target") then
                let fileDirUri = fileDir |> SpiralFileSystem.normalize_path |> SpiralFileSystem.new_file_uri
                let fileDeleteObj = {| FileDelete = {| uris = [| fileDirUri |] |} |}
                let! _fileDeleteResult = fileDeleteObj |> sendObj serverPort
                ()

            let outputPath = fileDir </> outputFileName
            return outputPath, result
    }
    loop 0

### SpiralInput

In [ ]:
type SpiralInput =
    | Spi of string * string option
    | Spir of string

### persistCode

In [ ]:
let persistCode (input: {| backend : Backend option; input: SpiralInput; packages: string [] |}) = async {
    let targetDir = workspaceRoot </> "target/spiral_Eval"

    let packagesDir = targetDir </> "packages"

    let hashHex = $"%A{input.backend}%A{input.input}" |> SpiralCrypto.hash_text

    let packageDir = packagesDir </> hashHex
    packageDir |> System.IO.Directory.CreateDirectory |> ignore

    let moduleName = "main"

    let spirModule, spiModule =
        match input.input with
        | Spi (_spi, Some _spir) -> true, true
        | Spi (_spi, None) -> false, true
        | Spir _spir -> true, false
        |> fun (spir, spi) ->
            (if spir then $"{moduleName}_real*-" else ""),
            if spi then moduleName else ""

    let libLinkTargetPath = workspaceRoot </> "../spiral/lib/spiral"
    let libLinkPath = packageDir </> "spiral"

    let packagesModule =
        input.packages
        |> Array.map (fun package ->
            let path = workspaceRoot </> package
            let packageName = path |> System.IO.Path.GetFileName
            let libLinkPath = packageDir </> packageName
            libLinkPath |> SpiralFileSystem.link_directory path
            $"{packageName}-"
        )
        |> String.concat "\n    "

    let packageDir' =
        if input.packages |> Array.isEmpty
        then workspaceRoot </> "../spiral/lib"
        else
            libLinkPath |> SpiralFileSystem.link_directory libLinkTargetPath
            packageDir

    let spiprojPath = packageDir </> "package.spiproj"
    let spiprojCode =
        $"""packageDir: {packageDir'}
packages:
    |core-
    spiral-
    {packagesModule}
modules:
    {spirModule}
    {spiModule}
"""
    do! spiprojCode |> SpiralFileSystem.write_all_text_exists spiprojPath

    let spirPath = packageDir </> $"{moduleName}_real.spir"
    let spiPath = packageDir </> $"{moduleName}.spi"

    let spirCode, spiCode =
        match input.input with
        | Spi (spi, Some spir) -> Some spir, Some spi
        | Spi (spi, None) -> None, Some spi
        | Spir spir -> Some spir, None

    match spirCode with
    | Some spir -> do! spir |> SpiralFileSystem.write_all_text_exists spirPath
    | None -> ()
    match spiCode with
    | Some spi -> do! spi |> SpiralFileSystem.write_all_text_exists spiPath
    | None -> ()

    let spiralPath =
        match input.input with
        | Spi _ -> spiPath
        | Spir _ -> spirPath
    match input.backend with
    | None -> return spiralPath, None
    | Some backend ->
        let outputFileName =
            let fileName =
                match input.input with
                | Spi _ -> moduleName
                | Spir _ -> $"{moduleName}_real"
            match backend with
            | Fsharp -> $"{fileName}.fsx"
            | Cuda -> $"{fileName}.py"
        let outputPath = packageDir </> outputFileName
        if outputPath |> System.IO.File.Exists |> not
        then return spiralPath, None
        else
            let! oldCode = spiralPath |> SpiralFileSystem.read_all_text_async
            if oldCode <> (spiCode |> Option.defaultValue (spirCode |> Option.defaultValue ""))
            then return spiralPath, None
            else
                let! outputCode = outputPath |> SpiralFileSystem.read_all_text_async
                return spiralPath, Some (outputPath, outputCode |> SpiralSm.replace "\r\n" "\n")
    }

### buildCode

In [ ]:
let buildCode backend packages isCache timeout cancellationToken input = async {
    let! mainPath, outputCache =
        persistCode {| input = input; backend = Some backend; packages = packages |}
    match outputCache with
    | Some (outputPath, outputCode) when isCache -> return mainPath, (outputPath, Some outputCode), []
    | _ ->
        let! outputPath, (outputCode, errors) = mainPath |> buildFile backend timeout None cancellationToken
        return mainPath, (outputPath, outputCode), errors
}

In [ ]:
//// test

"""inl app () =
    console.write_line "text"
    1i32

inl main () =
    app
    |> dyn
    |> ignore
"""
|> fun code -> Spi (code, None)
|> buildCode Fsharp [||] false 15000 None
|> Async.runWithTimeout 15000
|> Option.map (fun (_, (_, outputContent), errors) -> outputContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        Some """let rec closure1 () () : unit =
    let v0 : (string -> unit) = System.Console.WriteLine
    let v1 : string = "text"
    v0 v1
and closure0 () () : int32 =
    let v0 : unit = ()
    let v1 : (unit -> unit) = closure1()
    let v2 : unit = (fun () -> v1 (); v0) ()
    1
let v0 : (unit -> int32) = closure0()
()
""",
        []
    )
)

00:00:11 v #1 async.run_with_timeout_async / { timeout = 180 }
00:00:09 d #1 runtime.execute_with_options_async / { file_name = dotnet; arguments = US5_0
  ""c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"; options = { command = dotnet "c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64; cancellation_token = Some System.Threading.CancellationToken; environment_variables = [||]; on_line = Some <fun:compiler@22-4>; stdin = None; trace = true; working_directory = Some "c:\home\git\polyglot" } }
00:00:09 v #2 > 00:00:00 d #1 pwd: c:\home\git\polyglot
00:00:09 v #3 > 00:00:00 d #2 dllPath: c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:09 v #4 > 00:00:00 d #3 targetDir: c:\

In [ ]:
//// test

""
|> fun code -> Spi (code, None)
|> buildCode Fsharp [||] false 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (_, (_, outputContent), errors) -> outputContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        None,
        [ "Cannot find `main` in file main." ]
    )
)

00:00:18 v #6 async.run_with_timeout_async / { timeout = 180 }
00:00:15 d #7 runtime.execute_with_options_async / { file_name = dotnet; arguments = US5_0
  ""c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"; options = { command = dotnet "c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64; cancellation_token = Some System.Threading.CancellationToken; environment_variables = [||]; on_line = Some <fun:compiler@22-4>; stdin = None; trace = true; working_directory = Some "c:\home\git\polyglot" } }
00:00:15 v #8 > 00:00:00 d #1 pwd: c:\home\git\polyglot
00:00:15 v #9 > 00:00:00 d #2 dllPath: c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:15 v #10 > 00:00:00 d #3 targetDir: c:

In [ ]:
//// test

"""inl main () =
    1i32 / 0i32
"""
|> fun code -> Spi (code, None)
|> buildCode Fsharp [||] false 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (_, (_, outputContent), errors) -> outputContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        None,
        [ "An attempt to divide by zero has been detected at compile time." ]
    )
)

00:00:23 v #11 async.run_with_timeout_async / { timeout = 180 }
00:00:20 d #13 runtime.execute_with_options_async / { file_name = dotnet; arguments = US5_0
  ""c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"; options = { command = dotnet "c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64; cancellation_token = Some System.Threading.CancellationToken; environment_variables = [||]; on_line = Some <fun:compiler@22-4>; stdin = None; trace = true; working_directory = Some "c:\home\git\polyglot" } }
00:00:20 v #14 > 00:00:00 d #1 pwd: c:\home\git\polyglot
00:00:20 v #15 > 00:00:00 d #2 dllPath: c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:20 v #16 > 00:00:00 d #3 targetDir

In [ ]:
//// test

"""inl main () =
    1 + ""
"""
|> fun code -> Spi (code, None)
|> buildCode Fsharp [||] false 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (_, (_, outputContent), errors) -> outputContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        None,
        [
            "main.spi:
Constraint satisfaction error.
Got: string
Fails to satisfy: number"
        ]
    )
)

00:00:27 v #16 async.run_with_timeout_async / { timeout = 180 }
00:00:24 d #19 runtime.execute_with_options_async / { file_name = dotnet; arguments = US5_0
  ""c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"; options = { command = dotnet "c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64; cancellation_token = Some System.Threading.CancellationToken; environment_variables = [||]; on_line = Some <fun:compiler@22-4>; stdin = None; trace = true; working_directory = Some "c:\home\git\polyglot" } }
00:00:25 v #20 > 00:00:00 d #1 pwd: c:\home\git\polyglot
00:00:25 v #21 > 00:00:00 d #2 dllPath: c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:25 v #22 > 00:00:00 d #3 targetDir

In [ ]:
//// test

"""inl main () =
    x + y
"""
|> fun code -> Spi (code, None)
|> buildCode Fsharp [||] false 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (_, (_, outputContent), errors) -> outputContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        None,
        [
            "main.spi:
Unbound variable: x.
Unbound variable: y."
        ]
    )
)

00:00:32 v #21 async.run_with_timeout_async / { timeout = 180 }
00:00:29 d #25 runtime.execute_with_options_async / { file_name = dotnet; arguments = US5_0
  ""c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"; options = { command = dotnet "c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64; cancellation_token = Some System.Threading.CancellationToken; environment_variables = [||]; on_line = Some <fun:compiler@22-4>; stdin = None; trace = true; working_directory = Some "c:\home\git\polyglot" } }
00:00:29 v #26 > 00:00:00 d #1 pwd: c:\home\git\polyglot
00:00:29 v #27 > 00:00:00 d #2 dllPath: c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:29 v #28 > 00:00:00 d #3 targetDir

In [ ]:
//// test

"""
inl main () =
    real
        inl unbox_real forall a. (obj : a) : a =
            typecase obj with
            | _ => obj
        unbox_real ()
    ()
"""
|> fun code -> Spi (code, None)
|> buildCode Fsharp [||] false 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (_, (_, outputContent), errors) -> outputContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        None,
        [ "Cannot apply a forall with a term." ]
    )
)

00:00:36 v #27 async.run_with_timeout_async / { timeout = 180 }
00:00:33 d #31 runtime.execute_with_options_async / { file_name = dotnet; arguments = US5_0
  ""c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"; options = { command = dotnet "c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64; cancellation_token = Some System.Threading.CancellationToken; environment_variables = [||]; on_line = Some <fun:compiler@22-4>; stdin = None; trace = true; working_directory = Some "c:\home\git\polyglot" } }
00:00:33 v #32 > 00:00:00 d #1 pwd: c:\home\git\polyglot
00:00:33 v #33 > 00:00:00 d #2 dllPath: c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:33 v #34 > 00:00:00 d #3 targetDir

In [ ]:
//// test

"""
inl main () =
    real
        inl unbox_real forall a. (obj : a) : a =
            typecase obj with
            | _ => obj
        unbox_real `i32 1
"""
|> fun code -> Spi (code, None)
|> buildCode Fsharp [||] false 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (_, (_, outputContent), errors) -> outputContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        None,
        [ "The main function should not have a forall." ]
    )
)

00:00:40 v #32 async.run_with_timeout_async / { timeout = 180 }
00:00:38 d #37 runtime.execute_with_options_async / { file_name = dotnet; arguments = US5_0
  ""c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"; options = { command = dotnet "c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64; cancellation_token = Some System.Threading.CancellationToken; environment_variables = [||]; on_line = Some <fun:compiler@22-4>; stdin = None; trace = true; working_directory = Some "c:\home\git\polyglot" } }
00:00:38 v #38 > 00:00:00 d #1 pwd: c:\home\git\polyglot
00:00:38 v #39 > 00:00:00 d #2 dllPath: c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:38 v #40 > 00:00:00 d #3 targetDir

In [ ]:
//// test

"""
inl init_series start end inc =
    inl total : f64 = conv ((end - start) / inc) + 1
    listm.init total (conv >> (*) inc >> (+) start) : list f64

type integration = (f64 -> f64) -> f64 -> f64 -> f64

inl integral dt : integration =
    fun f a b =>
        init_series (a + dt / 2) (b - dt / 2) dt
        |> listm.map (f >> (*) dt)
        |> listm.fold (+) 0

inl main () =
    integral 0.1 (fun x => x ** 2) 0 1
"""
|> fun code -> Spi (code, None)
|> buildCode Fsharp [||] false 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (_, (_, outputContent), errors) -> outputContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        Some "0.3325000000000001\n",
        []
    )
)

00:00:45 v #37 async.run_with_timeout_async / { timeout = 180 }
00:00:43 d #43 runtime.execute_with_options_async / { file_name = dotnet; arguments = US5_0
  ""c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"; options = { command = dotnet "c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64; cancellation_token = Some System.Threading.CancellationToken; environment_variables = [||]; on_line = Some <fun:compiler@22-4>; stdin = None; trace = true; working_directory = Some "c:\home\git\polyglot" } }
00:00:43 v #44 > 00:00:00 d #1 pwd: c:\home\git\polyglot
00:00:43 v #45 > 00:00:00 d #2 dllPath: c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:43 v #46 > 00:00:00 d #3 targetDir

In [ ]:
//// test

"""
inl init_series start end inc =
    inl total : f64 = conv ((end - start) / inc) + 1
    listm.init total (conv >> (*) inc >> (+) start) : list f64

type integration = (f64 -> f64) -> f64 -> f64 -> f64

inl integral dt : integration =
    fun f a b =>
        init_series (a + dt / 2) (b - dt / 2) dt
        |> listm.map (f >> (*) dt)
        |> listm.fold (+) 0

inl main () =
    integral 0.1 (fun x => x ** 2) 0 1
"""
|> fun code -> Spi (code, None)
|> buildCode Cuda [||] false 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (_, (_, outputContent), errors) -> outputContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        Some @"kernel = r""""""
""""""
class static_array():
    def __init__(self, length):
        self.ptr = []
        for _ in range(length):
            self.ptr.append(None)

    def __getitem__(self, index):
        assert 0 <= index < len(self.ptr), ""The get index needs to be in range.""
        return self.ptr[index]
    
    def __setitem__(self, index, value):
        assert 0 <= index < len(self.ptr), ""The set index needs to be in range.""
        self.ptr[index] = value

class static_array_list(static_array):
    def __init__(self, length):
        super().__init__(length)
        self.length = 0

    def __getitem__(self, index):
        assert 0 <= index < self.length, ""The get index needs to be in range.""
        return self.ptr[index]
    
    def __setitem__(self, index, value):
        assert 0 <= index < self.length, ""The set index needs to be in range.""
        self.ptr[index] = value

    def push(self,value):
        assert (self.length < len(self.ptr)), ""The length before pushing has to be less than the maximum length of the array.""
        self.ptr[self.length] = value
        self.length += 1

    def pop(self):
        assert (0 < self.length), ""The length before popping has to be greater than 0.""
        self.length -= 1
        return self.ptr[self.length]

    def unsafe_set_length(self,i):
        assert 0 <= i <= len(self.ptr), ""The new length has to be in range.""
        self.length = i

class dynamic_array(static_array): 
    pass

class dynamic_array_list(static_array_list):
    def length_(self): return self.length

import cupy as cp
import numpy as np
from dataclasses import dataclass
from typing import NamedTuple, Union, Callable, Tuple
i8 = int; i16 = int; i32 = int; i64 = int; u8 = int; u16 = int; u32 = int; u64 = int; f32 = float; f64 = float; char = str; string = str
cuda = False

def main_body():
    return 0.3325000000000001

def main():
    r = main_body()
    if cuda: cp.cuda.get_current_stream().synchronize() # This line is here so the `__trap()` calls on the kernel aren't missed.
    return r

if __name__ == '__main__': result = main(); None if result is None else print(result)
",
        []
    )
)

00:00:50 v #41 async.run_with_timeout_async / { timeout = 180 }
00:00:47 d #49 runtime.execute_with_options_async / { file_name = dotnet; arguments = US5_0
  ""c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"; options = { command = dotnet "c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64; cancellation_token = Some System.Threading.CancellationToken; environment_variables = [||]; on_line = Some <fun:compiler@22-4>; stdin = None; trace = true; working_directory = Some "c:\home\git\polyglot" } }
00:00:48 v #50 > 00:00:00 d #1 pwd: c:\home\git\polyglot
00:00:48 v #51 > 00:00:00 d #2 dllPath: c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:48 v #52 > 00:00:00 d #3 targetDir

In [ ]:
//// test

"""
inl init_series start end inc =
    inl total : f64 = conv ((end - start) / inc) + 1
    listm.init total (conv >> (*) inc >> (+) start) : list f64

type integration = (f64 -> f64) -> f64 -> f64 -> f64

inl integral dt : integration =
    fun f a b =>
        init_series (a + dt / 2) (b - dt / 2) dt
        |> listm.map (f >> (*) dt)
        |> listm.fold (+) 0

inl main () =
    integral 0.01 (fun x => x ** 2) 0 1
"""
|> fun code -> Spi (code, None)
|> buildCode Fsharp [||] false 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (_, (_, outputContent), errors) -> outputContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        Some "0.33332500000000004\n",
        []
    )
)
// |> _assertEqual None
// |> fun x -> printfn $"{x.ToDisplayString ()}"

00:00:55 v #46 async.run_with_timeout_async / { timeout = 180 }
00:00:52 d #55 runtime.execute_with_options_async / { file_name = dotnet; arguments = US5_0
  ""c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"; options = { command = dotnet "c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64; cancellation_token = Some System.Threading.CancellationToken; environment_variables = [||]; on_line = Some <fun:compiler@22-4>; stdin = None; trace = true; working_directory = Some "c:\home\git\polyglot" } }
00:00:52 v #56 > 00:00:00 d #1 pwd: c:\home\git\polyglot
00:00:52 v #57 > 00:00:00 d #2 dllPath: c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:52 v #58 > 00:00:00 d #3 targetDir

In [ ]:
//// test

"""inl rec main () = main"""
|> fun code -> Spi (code, None)
|> buildCode Fsharp [||] false 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (_, (_, outputContent), errors) -> outputContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        None,
        [
            "main.spi:
Recursive metavariables are not allowed. A metavar cannot be unified with a type that has itself.
Got:      'a
Expected: () -> 'a"
        ]
    )
)
// |> _assertEqual None
// |> fun x -> printfn $"{x.ToDisplayString ()}"

00:00:59 v #51 async.run_with_timeout_async / { timeout = 180 }
00:00:57 d #61 runtime.execute_with_options_async / { file_name = dotnet; arguments = US5_0
  ""c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"; options = { command = dotnet "c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64; cancellation_token = Some System.Threading.CancellationToken; environment_variables = [||]; on_line = Some <fun:compiler@22-4>; stdin = None; trace = true; working_directory = Some "c:\home\git\polyglot" } }
00:00:57 v #62 > 00:00:00 d #1 pwd: c:\home\git\polyglot
00:00:57 v #63 > 00:00:00 d #2 dllPath: c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:57 v #64 > 00:00:00 d #3 targetDir

## getFileTokenRange

In [ ]:
let getFileTokenRange port cancellationToken path = async {
    let fullPath = path |> System.IO.Path.GetFullPath
    let! code = fullPath |> SpiralFileSystem.read_all_text_async
    let lines = code |> SpiralSm.split "\n"

    let struct (token, disposable) = SpiralThreading.new_disposable_token cancellationToken
    use _ = disposable

    let port = port |> Option.defaultWith getCompilerPort
    let! serverPort, _errors, ct, disposable = awaitCompiler port (Some token)
    use _ = disposable

    let fullPathUri = fullPath |> SpiralFileSystem.normalize_path |> SpiralFileSystem.new_file_uri

    let fileOpenObj = {| FileOpen = {| uri = fullPathUri; spiText = code |} |}
    let! _fileOpenResult = fileOpenObj |> sendObj serverPort

    // do! Async.Sleep 60

    let fileTokenRangeObj =
        {|
            FileTokenRange =
                {|
                    uri = fullPathUri
                    range =
                        [|
                            {|
                                line = 0
                                character = 0
                            |}
                            {|
                                line = lines.Length - 1
                                character = lines.[lines.Length - 1].Length
                            |}
                        |]
                |}
        |}
    let! fileTokenRangeResult =
        fileTokenRangeObj
        |> sendObj serverPort
        |> Async.withCancellationToken ct

    let fileDir = fullPath |> System.IO.Path.GetDirectoryName
    if fileDir |> SpiralSm.starts_with (workspaceRoot </> "target") then
        let fileDirUri = fileDir |> SpiralFileSystem.normalize_path |> SpiralFileSystem.new_file_uri
        let fileDeleteObj = {| FileDelete = {| uris = [| fileDirUri |] |} |}
        let! _fileDeleteResult = fileDeleteObj |> sendObj serverPort
        ()

    return fileTokenRangeResult |> Option.map FSharp.Json.Json.deserialize<int array>
}

## getCodeTokenRange

In [ ]:
let getCodeTokenRange cancellationToken code = async {
    let! mainPath, _ =
        persistCode {| input = Spi (code, None); backend = None; packages = [||] |}

    let codeDir = mainPath |> System.IO.Path.GetDirectoryName
    let tokensPath = codeDir </> "tokens.json"
    let! tokens = async {
        if tokensPath |> System.IO.File.Exists |> not
        then return None
        else
            let! text = tokensPath |> SpiralFileSystem.read_all_text_async

            return
                if text.Length > 2
                then text |> FSharp.Json.Json.deserialize<int array> |> Some
                else None
    }
    match tokens with
    | Some tokens ->
        return tokens |> Some
    | None -> return! mainPath |> getFileTokenRange None cancellationToken
}

In [ ]:
//// test

"""inl main () = ()"""
|> getCodeTokenRange None
|> Async.runWithTimeout 10000
|> Option.flatten
|> _assertEqual (Some [| 0; 0; 3; 7; 0; 0; 4; 4; 0; 0; 0; 5; 1; 8; 0; 0; 1; 1; 8; 0; 0; 2; 1; 4; 0; 0;
2; 1; 8; 0; 0; 1; 1; 8; 0 |])

00:01:04 v #57 async.run_with_timeout_async / { timeout = 180 }
00:01:01 d #67 runtime.execute_with_options_async / { file_name = dotnet; arguments = US5_0
  ""c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"; options = { command = dotnet "c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64; cancellation_token = Some System.Threading.CancellationToken; environment_variables = [||]; on_line = Some <fun:compiler@22-4>; stdin = None; trace = true; working_directory = Some "c:\home\git\polyglot" } }
00:01:02 v #68 > 00:00:00 d #1 pwd: c:\home\git\polyglot
00:01:02 v #69 > 00:00:00 d #2 dllPath: c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:01:02 v #70 > 00:00:00 d #3 targetDir

In [ ]:
//// test

"""inl main () = 1i32"""
|> getCodeTokenRange None
|> Async.runWithTimeout 10000
|> Option.flatten
|> _assertEqual (Some [| 0; 0; 3; 7; 0; 0; 4; 4; 0; 0; 0; 5; 1; 8; 0; 0; 1; 1; 8; 0; 0; 2; 1; 4; 0; 0;
2; 1; 3; 0; 0; 1; 3; 12; 0 |])

00:01:06 v #62 async.run_with_timeout_async / { timeout = 180 }
00:01:04 d #73 runtime.execute_with_options_async / { file_name = dotnet; arguments = US5_0
  ""c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"; options = { command = dotnet "c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64; cancellation_token = Some System.Threading.CancellationToken; environment_variables = [||]; on_line = Some <fun:compiler@22-4>; stdin = None; trace = true; working_directory = Some "c:\home\git\polyglot" } }
00:01:04 v #74 > 00:00:00 d #1 pwd: c:\home\git\polyglot
00:01:04 v #75 > 00:00:00 d #2 dllPath: c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:01:04 v #76 > 00:00:00 d #3 targetDir

## getFileHoverAt

In [ ]:
let getFileHoverAt
    port
    cancellationToken
    path
    (position : {| line: int; character: int |})
    = async {
    let fullPath = path |> System.IO.Path.GetFullPath
    let! code = fullPath |> SpiralFileSystem.read_all_text_async
    let lines = code |> SpiralSm.split "\n"

    let struct (token, disposable) = SpiralThreading.new_disposable_token cancellationToken
    use _ = disposable

    let port = port |> Option.defaultWith getCompilerPort
    let! serverPort, _errors, ct, disposable = awaitCompiler port (Some token)
    use _ = disposable

    let fullPathUri = fullPath |> SpiralFileSystem.normalize_path |> SpiralFileSystem.new_file_uri

    let fileOpenObj = {| FileOpen = {| uri = fullPathUri; spiText = code |} |}
    let! _fileOpenResult = fileOpenObj |> sendObj serverPort

    // do! Async.Sleep 60

    let hoverAtObj =
        {|
            HoverAt =
                {|
                    uri = fullPathUri
                    pos = position
                |}
        |}
    let! hoverAtResult =
        hoverAtObj
        |> sendObj serverPort
        |> Async.withCancellationToken ct

    let fileDir = fullPath |> System.IO.Path.GetDirectoryName
    if fileDir |> SpiralSm.starts_with (workspaceRoot </> "target") then
        let fileDirUri = fileDir |> SpiralFileSystem.normalize_path |> SpiralFileSystem.new_file_uri
        let fileDeleteObj = {| FileDelete = {| uris = [| fileDirUri |] |} |}
        let! _fileDeleteResult = fileDeleteObj |> sendObj serverPort
        ()

    return hoverAtResult
}

## getCodeHoverAt

In [ ]:
let getCodeHoverAt cancellationToken code position = async {
    let! mainPath, _ =
        persistCode {| input = Spi (code, None); backend = None; packages = [||] |}

    let codeDir = mainPath |> System.IO.Path.GetDirectoryName
    let filePath = codeDir </> "hover.json"
    let! output = async {
        if filePath |> System.IO.File.Exists |> not
        then return None
        else
            let! text = filePath |> SpiralFileSystem.read_all_text_async

            return
                if text.Length > 2
                then text |> Some
                else None
    }
    match output with
    | Some output ->
        return output |> Some
    | None -> return! getFileHoverAt None cancellationToken mainPath position
}

In [ ]:
//// test

getCodeHoverAt None """inl main () = ()""" {| line = 0; character = 4 |}
|> Async.runWithTimeout 10000
|> Option.flatten
|> _assertEqual (Some "() -> ()")

00:01:09 v #67 async.run_with_timeout_async / { timeout = 180 }
00:01:06 d #79 runtime.execute_with_options_async / { file_name = dotnet; arguments = US5_0
  ""c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"; options = { command = dotnet "c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64; cancellation_token = Some System.Threading.CancellationToken; environment_variables = [||]; on_line = Some <fun:compiler@22-4>; stdin = None; trace = true; working_directory = Some "c:\home\git\polyglot" } }
00:01:07 v #80 > 00:00:00 d #1 pwd: c:\home\git\polyglot
00:01:07 v #81 > 00:00:00 d #2 dllPath: c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:01:07 v #82 > 00:00:00 d #3 targetDir

In [ ]:
//// test

getCodeHoverAt None """inl main () = ()""" {| line = 0; character = 0 |}
|> Async.runWithTimeout 10000
|> Option.flatten
|> _assertEqual (Some null)

00:01:13 v #72 async.run_with_timeout_async / { timeout = 180 }
00:01:10 d #85 runtime.execute_with_options_async / { file_name = dotnet; arguments = US5_0
  ""c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"; options = { command = dotnet "c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64; cancellation_token = Some System.Threading.CancellationToken; environment_variables = [||]; on_line = Some <fun:compiler@22-4>; stdin = None; trace = true; working_directory = Some "c:\home\git\polyglot" } }
00:01:11 v #86 > 00:00:00 d #1 pwd: c:\home\git\polyglot
00:01:11 v #87 > 00:00:00 d #2 dllPath: c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:01:11 v #88 > 00:00:00 d #3 targetDir

In [ ]:
//// test

getCodeHoverAt None """inl rec main () = main""" {| line = 0; character = 8 |}
|> Async.runWithTimeout 10000
|> Option.flatten
|> _assertEqual (Some "forall 'a. () -> 'a")

00:01:17 v #77 async.run_with_timeout_async / { timeout = 180 }
00:01:14 d #91 runtime.execute_with_options_async / { file_name = dotnet; arguments = US5_0
  ""c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"; options = { command = dotnet "c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64; cancellation_token = Some System.Threading.CancellationToken; environment_variables = [||]; on_line = Some <fun:compiler@22-4>; stdin = None; trace = true; working_directory = Some "c:\home\git\polyglot" } }
00:01:15 v #92 > 00:00:00 d #1 pwd: c:\home\git\polyglot
00:01:15 v #93 > 00:00:00 d #2 dllPath: c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:01:15 v #94 > 00:00:00 d #3 targetDir

In [ ]:
//// test

getCodeHoverAt None """inl main () = 1""" {| line = 0; character = 4 |}
|> Async.runWithTimeout 10000
|> Option.flatten
|> _assertEqual (Some "forall 'a {number}. () -> 'a")

00:01:21 v #82 async.run_with_timeout_async / { timeout = 180 }
00:01:18 d #97 runtime.execute_with_options_async / { file_name = dotnet; arguments = US5_0
  ""c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"; options = { command = dotnet "c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64; cancellation_token = Some System.Threading.CancellationToken; environment_variables = [||]; on_line = Some <fun:compiler@22-4>; stdin = None; trace = true; working_directory = Some "c:\home\git\polyglot" } }
00:01:19 v #98 > 00:00:00 d #1 pwd: c:\home\git\polyglot
00:01:19 v #99 > 00:00:00 d #2 dllPath: c:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:01:19 v #100 > 00:00:00 d #3 targetDi

## Arguments

In [ ]:
[<RequireQualifiedAccess>]
type Arguments =
    | Build_File of string * string
    | File_Token_Range of string * string
    | File_Hover_At of string * string * int * int
    | Execute_Command of string
    | [<Argu.ArguAttributes.Unique>] Timeout of int
    | [<Argu.ArguAttributes.Unique>] Port of int
    | [<Argu.ArguAttributes.Unique>] Parallel
    | [<Argu.ArguAttributes.Unique>] Exit_On_Error

    interface Argu.IArgParserTemplate with
        member s.Usage =
            match s with
            | Build_File _ -> nameof Build_File
            | File_Token_Range _ -> nameof File_Token_Range
            | File_Hover_At _ -> nameof File_Hover_At
            | Execute_Command _ -> nameof Execute_Command
            | Timeout _ -> nameof Timeout
            | Port _ -> nameof Port
            | Parallel -> nameof Parallel
            | Exit_On_Error-> nameof Exit_On_Error

In [ ]:
//// test

Argu.ArgumentParser.Create<Arguments>().PrintUsage ()

"USAGE: dotnet-repl [--help] [--build-file <string> <string>]
                   [--file-token-range <string> <string>]
                   [--file-hover-at <string> <string> <int> <int>]
                   [--execute-command <string>] [--timeout <int>] [--port <int>]
                   [--parallel] [--exit-on-error]

OPTIONS:

    --build-file <string> <string>
                          Build_File
    --file-token-range <string> <string>
                          File_Token_Range
    --file-hover-at <string> <string> <int> <int>
                          File_Hover_At
    --execute-command <string>
                          Execute_Command
    --timeout <int>       Timeout
    --port <int>          Port
    --parallel            Parallel
    --exit-on-error       Exit_On_Error
    --help                display this list of options.
"


## main

In [ ]:
let main args =
    let argsMap = args |> Runtime.parseArgsMap<Arguments>

    let buildFileActions =
        argsMap
        |> Map.tryFind (nameof Arguments.Build_File)
        |> Option.defaultValue []
        |> List.choose (function
            | Arguments.Build_File (inputPath, outputPath) -> Some (inputPath, outputPath)
            | _ -> None
        )

    let fileTokenRangeActions =
        argsMap
        |> Map.tryFind (nameof Arguments.File_Token_Range)
        |> Option.defaultValue []
        |> List.choose (function
            | Arguments.File_Token_Range (inputPath, outputPath) -> Some (inputPath, outputPath)
            | _ -> None
        )

    let fileHoverAtActions =
        argsMap
        |> Map.tryFind (nameof Arguments.File_Hover_At)
        |> Option.defaultValue []
        |> List.choose (function
            | Arguments.File_Hover_At (inputPath, outputPath, line, character) ->
                Some (inputPath, outputPath, line, character)
            | _ -> None
        )

    let executeCommandActions =
        argsMap
        |> Map.tryFind (nameof Arguments.Execute_Command)
        |> Option.defaultValue []
        |> List.choose (function
            | Arguments.Execute_Command command -> Some command
            | _ -> None
        )

    let timeout =
        match argsMap |> Map.tryFind (nameof Arguments.Timeout) with
        | Some [ Arguments.Timeout timeout ] -> timeout
        | _ -> 60000 * 60

    let port =
        match argsMap |> Map.tryFind (nameof Arguments.Port) with
        | Some [ Arguments.Port port ] -> Some port
        | _ -> None

    let isParallel = argsMap |> Map.containsKey (nameof Arguments.Parallel)

    let isExitOnError = argsMap |> Map.containsKey (nameof Arguments.Exit_On_Error)

    async {
        let port =
            port
            |> Option.defaultWith getCompilerPort
        let struct (localToken, disposable) = SpiralThreading.new_disposable_token None
        let! serverPort, _errors, compilerToken, disposable = awaitCompiler port (Some localToken)
        use _ = disposable

        let buildFileAsync =
            buildFileActions
            |> List.map (fun (inputPath, outputPath) -> async {
                let! _outputPath, (outputCode, errors) =
                    let backend =
                        if outputPath |> SpiralSm.ends_with ".fsx"
                        then Fsharp
                        elif outputPath |> SpiralSm.ends_with ".py"
                        then Cuda
                        else failwith $"Supervisor.main / invalid backend / outputPath: {outputPath}"
                    let isReal = inputPath |> SpiralSm.ends_with ".spir"
                    inputPath |> buildFile backend timeout (Some serverPort) None

                errors
                |> List.map snd
                |> List.iter (fun error ->
                    trace Critical (fun () -> $"main / error: {error |> serializeObj}") _locals
                )

                match outputCode with
                | Some outputCode ->
                    do! outputCode |> SpiralFileSystem.write_all_text_exists outputPath
                    return 0
                | None ->
                    if isExitOnError
                    then SpiralRuntime.current_process_kill ()

                    return 1
            })

        let fileTokenRangeAsync =
            fileTokenRangeActions
            |> List.map (fun (inputPath, outputPath) -> async {
                let! tokenRange = inputPath |> getFileTokenRange (Some serverPort) None
                match tokenRange with
                | Some tokenRange ->
                    do! tokenRange |> FSharp.Json.Json.serialize |> SpiralFileSystem.write_all_text_exists outputPath
                    return 0
                | None ->
                    if isExitOnError
                    then SpiralRuntime.current_process_kill ()

                    return 1
            })

        let fileHoverAtAsync =
            fileHoverAtActions
            |> List.map (fun (inputPath, outputPath, line, character) -> async {
                let! hoverAt =
                    getFileHoverAt
                        (Some serverPort)
                        None
                        inputPath
                        {| line = line; character = character |}
                match hoverAt with
                | Some hoverAt ->
                    do! hoverAt |> FSharp.Json.Json.serialize |> SpiralFileSystem.write_all_text_exists outputPath
                    return 0
                | None ->
                    if isExitOnError
                    then SpiralRuntime.current_process_kill ()

                    return 1
            })

        let executeCommandAsync =
            executeCommandActions
            |> List.map (fun command -> async {
                let! exitCode, result =
                    SpiralRuntime.execution_options (fun x ->
                        { x with
                            l0 = command
                            l1 = Some compilerToken
                        }
                    )
                    |> SpiralRuntime.execute_with_options_async

                trace Debug (fun () -> $"main / executeCommand / exitCode: {exitCode} / command: {command}") _locals

                if isExitOnError && exitCode <> 0
                then SpiralRuntime.current_process_kill ()

                return exitCode
            })

        return!
            [| buildFileAsync; fileTokenRangeAsync; fileHoverAtAsync; executeCommandAsync |]
            |> Seq.collect id
            |> fun x ->
                if isParallel
                then Async.Parallel (x, float System.Environment.ProcessorCount * 0.51 |> ceil |> int)
                else Async.Sequential x
            |> Async.map Array.sum
    }
    |> Async.runWithTimeout timeout
    |> Option.defaultValue 1

In [ ]:
//// test

let args =
    System.Environment.GetEnvironmentVariable "ARGS"
    |> SpiralRuntime.split_args
    |> Result.toArray
    |> Array.collect id

match args with
| [||] -> 0
| args -> if main args = 0 then 0 else failwith "main failed"

0